In [253]:
from enum import Enum
from collections import deque

class Move(Enum):
    FORWARD = 0
    DOWN = 1
    UP = 2

class Submarine:
    def __init__(self):
        self._depth = 0
        self._distance = 0
        self._aim = 0
        self._diagnostic = None

    def move_forward(self, magnitude):
        self._distance += magnitude
        self._depth += self._aim * magnitude

    def move_up(self, magnitude):
        self._aim -= magnitude
        
    def move_down(self, magnitude):
        self._aim += magnitude

    def move(self, instructions):
        move = {Move.FORWARD: self.move_forward,
                Move.DOWN: self.move_down,
                Move.UP: self.move_up}
        for instruction in instructions:
            direction = instruction[0]
            magnitude = instruction[1]
            move[direction](magnitude)

    def dephtXdistance(self):
        return self._depth * self._distance

    def read_diagnostic_report(self, report_path):
        self._diagnostic = Diagnostic(report_path)

    def get_power_consumption(self):
        if self._diagnostic:
            return self._diagnostic.get_power_consumption()
        else:
            return -1

    def get_life_support_rating(self):
        if self._diagnostic:
            return self._diagnostic.get_life_support_rating()
        else:
            return -1

    @staticmethod
    def get_instructions(file_path):
        instructions = []
        with open(file_path) as f:
            for line in f.readlines():
                move = line.rstrip('\n').split()
                if move[0] == 'forward':
                    direction = Move.FORWARD
                elif move[0] == 'down':
                    direction = Move.DOWN
                else:
                    direction = Move.UP
                instructions.append([direction, int(move[1])])
        return instructions

class Diagnostic:
    def __init__(self, file_path):
        self._report = []
        self._mcb = None   # Most common bit
        self._lcb = None   # Least common bit
        self._set_report(file_path)

    def get_power_consumption(self):
        gamma_rate = int(''.join(self._mcb), 2)
        epsilon_rate = int(''.join(self._lcb), 2)
        return gamma_rate * epsilon_rate

    def get_life_support_rating(self):
        mcb_match = lambda bit_array, idx: '1' if bit_array[idx] >= 0 else '0'
        lcb_match = lambda bit_array, idx: '1' if bit_array[idx] < 0 else '0'
        oxygen_report = self._filter_by_bit_match(self._report, mcb_match)
        co2_report = self._filter_by_bit_match(self._report, lcb_match)
        return int(oxygen_report[0], 2) * int(co2_report[0], 2)

    def _set_report(self, file_path):
        with open(file_path) as f:
            for line in f.readlines():
                self._report.append(line.rstrip('\n'))
        self._calculate_common_bits()

    def _calculate_common_bits(self):
        bit_array = self._create_common_bits_array(self._report)
        self._mcb = ['1' if x >= 0 else '0' for x in bit_array]
        self._lcb = ['1' if x < 0 else '0' for x in bit_array]
        
    @staticmethod
    def _create_common_bits_array(report):
        record_size = len(report[0])
        bit_array = [0] * record_size
        for record in report:
            for i in range(record_size):
                bit_array[i] += 1 if record[i] == '1' else -1
        return bit_array

    @staticmethod
    def _remove_bit_match(idx, bit_match, report):
        for i in range(len(report) -1, -1, -1):
            if report[i][idx] == bit_match:
                del report[i]
        return report

    def _filter_by_bit_match(self, report, bit_match_function):
        report_cpy = list(report)
        idx = 0
        while (len(report_cpy) > 1):
            bit_array = self._create_common_bits_array(report_cpy)
            bit_match = bit_match_function(bit_array, idx)
            report_cpy = self._remove_bit_match(idx, bit_match, report_cpy)
            idx += 1
        return report_cpy

class Bingo:
    GRID_SIZE = 5
    def __init__(self, input_path):
        self._draws = []
        self._boards = []
        self._board_winners = 0
        self._load_bingo(input_path)
    
    def _load_bingo(self, input_path):
        with open(input_path) as f:
            self._draws = [int(i) for i in f.readline().rstrip('\n').split(',')]
            f.readline()    # empty line
            end_of_file = False
            while(not end_of_file):
                board_array = []
                for i in range(self.GRID_SIZE):
                    line = f.readline()
                    if not line:
                        end_of_file = True
                        break
                    board_array.append([int(i) for i in line.rstrip('\n').split()])
                f.readline()  # empty line 
                if not end_of_file:
                    self._boards.append(self.BingoBoard(board_array))

    def play_game(self):
        for draw in self._draws:
            for board in self._boards:
                board.check(draw)
                if board.is_winner():                   
                    return board, board.score()
        return -1

    def get_last_winner(self):
        self._reset_game()
        boards_queue = list(self._boards)
        draw_idx = 0
        board = None
        while (len(boards_queue) > 0):
            next_number = self._draws[draw_idx]
            draw_idx += 1
            # Walk array back to remove elements
            for i in range(len(boards_queue) -1, -1, -1):
                board = boards_queue[i]
                board.check(next_number)
                if board.is_winner():
                    boards_queue.pop(i)
        return board, board.score()

    def _reset_game(self):
        for board in self._boards:
            board.reset()

    class BingoBoard:
        def __init__(self, board_array):
            self._size = len(board_array)
            self._board = board_array
            self._mark_matrix = []
            self.reset()
            
            self._row_sum = [0] * self._size
            self._col_sum = [0] * self._size
            self._last_number = -1

        def check(self, value):
            for row in range(self._size):
                for col in range (self._size):
                    if self._board[row][col] == value:
                        self._mark_matrix[row][col] = 1
                        self._row_sum[row] += 1
                        self._col_sum[col] += 1
                        self._last_number = value

        def is_winner(self):
            for row_sum in self._row_sum:
                if row_sum == self._size:
                    return True
            for col_sum in self._col_sum:
                if col_sum == self._size:
                    return True            
            return False

        def get_marks(self):
            return self._mark_matrix

        def score(self):
            sum = 0
            for row in range(self._size):
                for col in range(self._size):
                    if self._mark_matrix[row][col] == 0:
                        sum += self._board[row][col]
            return sum * self._last_number

        def reset(self):
            self._row_sum = [0] * self._size
            self._col_sum = [0] * self._size
            self._last_number = -1
            self._mark_matrix = []
            for i in range(self._size):
                self._mark_matrix.append([0] * self._size)

        def __str__(self):
            return str(self._board)

        def __repr__(self):
            return str(self)




In [254]:

submarine = Submarine()
instructions = Submarine.get_instructions('input.txt')
submarine.move(instructions)
print('Advent of code day2, submarine position:', submarine.dephtXdistance())

submarine.read_diagnostic_report('diagnostic_report.txt')
print('Day 3, Power Consumption:', submarine.get_power_consumption())
print('Day 3, Life support rating:', submarine.get_life_support_rating())

bingo = Bingo('bingo.txt')
board_winner, score = bingo.play_game()
board_last, last_score = bingo.get_last_winner()
print('Day 4, Bingo score:', score)
print('Day 4, Bingo Last Winner score:', last_score)

Advent of code day2, submarine position: 1960569556
Day 3, Power Consumption: 775304
Day 3, Life support rating: 1370737
Day 4, Bingo score: 33462
Day 4, Bingo Last Winner score: 30070
